<a href="https://colab.research.google.com/github/shilf1/study/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 기본 설정 테스트

In [1]:
!python --version

Python 3.7.13


In [2]:
import keras
keras.__version__

'2.8.0'

In [3]:
!nvidia-smi

Sun Apr 17 12:48:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from google.colab import drive
drive.mount("/drive")

Mounted at /drive


In [5]:
!ls "/drive/My Drive/2_study/caltech101"

accordion     cougar_face      gerenuk	     metronome	 soccer_ball
airplanes     crab	       gramophone    minaret	 stapler
anchor	      crayfish	       grand_piano   Motorbikes  starfish
ant	      crocodile        hawksbill     nautilus	 stegosaurus
barrel	      crocodile_head   headphone     octopus	 stop_sign
bass	      cup	       hedgehog      okapi	 strawberry
beaver	      dalmatian        helicopter    pagoda	 sunflower
binocular     dollar_bill      ibis	     panda	 tick
bonsai	      dolphin	       inline_skate  pigeon	 trilobite
brain	      dragonfly        joshua_tree   pizza	 umbrella
brontosaurus  electric_guitar  kangaroo      platypus	 watch
buddha	      elephant	       ketch	     pyramid	 water_lilly
butterfly     emu	       lamp	     revolver	 wheelchair
camera	      euphonium        laptop	     rhino	 wild_cat
cannon	      ewer	       Leopards      rooster	 windsor_chair
car_side      Faces	       llama	     saxophone	 wrench
ceiling_fan   Faces_easy       lobster	     

# 학습 데이터셋 준비 및 이미지 전처리

In [6]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1. / 255, validation_split=0.2)

In [7]:
batch_size = 16
input_image_size = (200, 200)

train_generator = train_datagen.flow_from_directory(
    '/drive/My Drive/2_study/caltech101',
    target_size=input_image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

Found 6986 images belonging to 101 classes.


In [8]:
validation_generator = train_datagen.flow_from_directory(
    '/drive/My Drive/2_study/caltech101',
    target_size=input_image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

Found 1696 images belonging to 101 classes.


# CNN 모델 구축

In [12]:
from keras.layers import Dense, Input, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.models import Model, Sequential

In [13]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))

In [14]:
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

In [15]:
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(101))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 198, 198, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 99, 99, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 97, 97, 32)        9248      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 48, 48, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 46, 46, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 23, 23, 64)      

# dataset 학습

In [16]:
import tensorflow as tf

model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['acc']
)

In [ ]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(patience=15, mode = 'auto', monitor='val_acc')
hist = model.fit_generator(train_generator,
                           steps_per_epoch=train_generator.samples //batch_size,
                           validation_data = validation_generator,
                           epochs=100,
                           callbacks=[early_stopping]
                           )


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/100
 47/436 [==>...........................] - ETA: 21:08 - loss: 4.5327 - acc: 0.1489

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'g', label='validation loss')
acc_ax.plot(hist.history['acc'], 'b', label='train acc')
acc_ax.plot(hist.history['val_acc'], 'r', label='validation acc')

plt.legend(['train acc', 'validation acc'])
plt.show()